In [1]:
import pandas as pd
import statistics

In [2]:
# define experiment to be preprocessed
experiment_series = "2024_09_17"

In [3]:
# Define upstream locations for each sampling point
upstream_1 = ["00083", "00084", "00085"]
upstream_2 = ["00102", "00103"] 
upstream_3 = upstream_1 + upstream_2 + ["00086", "00087"] 
upstream_4 = upstream_3 + ["00088", "00089", "00090", "00091", "00092", "00093", "00094", "00095", "00096", "00097", "00098", "00099", "00100", "00101"]
upstream_5 = ["00111", "00112", "00113", "00114", "00115", "00116", "00117", "00118", "00119", "00120", "00121", "00122", "00123", "00124", "00125", "00001", "00002", "00003", "00004", "00005", "00006"]
upstream_6 = ["00033", "00034", "00035", "00036", "00037", "00038", "00039", "00104", "00105", "00106", "00107", "00108", "00109", "00110"]
upstream_7 = upstream_4 + upstream_5 + upstream_6 + ["00007", "00008", "00009", "00010"]
upstream_8 = upstream_7 + ["00011", "00012", "00013", "00014"]
upstream_9 = ["00033", "00034", "00035", "00036", "00037", "00038", "00039"]
upstream_10 = upstream_9 + ["00040", "00041", "00042", "00043"]
upstream_11 = upstream_10 + ["00079", "00080", "00081", "00082"]
upstream_12 = upstream_8 + upstream_11 + ["00015", "00016", "00017", "00018"]
upstream_13 = ["00070", "00072", "00074", "00075", "00076", "00077", "00078"]
upstream_14 = upstream_12 + ["00019", "00020", "00021", "00022", "00023", "00024", "00025", "00026", "00027", "00028"]
upstream_15 = upstream_10 + ["00044", "00045", "00046", "00047", "00048", "00049", "00050", "00051", "00052", "00053", "00054", "00055", "00056", "00057", "00058", "00059", "00060", "00061", "00062", "00063", "00064", "00066", "00068", "00031"]
upstream_16 = upstream_13 + upstream_14 + upstream_15 + ["00029", "00030", "00032", "00069", "00067", "00065"]

upstream_locations_dict = {"1": upstream_1, "2": upstream_2, "3": upstream_3, "4": upstream_4, "5": upstream_5, "6": upstream_6, "7": upstream_7, "8": upstream_8, 
                           "9": upstream_9, "10": upstream_10, "11": upstream_11, "12": upstream_12, "13": upstream_13, "14": upstream_14, "15": upstream_15, "16": upstream_16}

In [4]:
def get_systems_location(df_locations, memilio_location):
    locs = df_locations.loc[df_locations.memilio_locations.apply(lambda locations: memilio_location in locations), "systems_location"]
    return locs.iloc[0]

def daily_agents_one_loc(row_output):
        daily_agents = []
        output = row_output[0].split(" ")[2:]
        
        for current_day in range(0, 14):
            agents = []
            for t in range(0, 24):
                current_n_agents = int(output[1])
                n_entries = current_n_agents*2
                timestep_output = output[2:2+n_entries]
                timestep_output = [float(i) for i in timestep_output]
                current_agents = timestep_output[::2]
                agents.append(current_agents)
                output = output[2+n_entries:]
            agents = [x for xs in agents for x in xs]
            daily_agents.append(list(set(agents)))
        
        return daily_agents

In [9]:
results = []
for sim in range(1, 251):
    df_locations = pd.read_csv(f"../../simulation_output/infection_model/{experiment_series}/{sim}_location_mapping.txt", index_col=None, header=None)
    df_locations["systems_location"] = df_locations.apply(lambda row: row.iloc[0].split(" ")[0].zfill(5), axis=1)
    df_locations["memilio_locations"] = df_locations.apply(lambda row: row.iloc[0].split(" ")[1:], axis=1)
    df_locations["memilio_locations"] = df_locations.memilio_locations.apply(lambda x: [i for i in x if i != ""])
    df_locations = df_locations[['systems_location', 'memilio_locations']]
    
    df = pd.read_csv(f"../../simulation_output/infection_model/{experiment_series}/{sim}_output.txt", header=None)
    df["memilio_location"] = df.apply(lambda row: row.iloc[0].split(" ")[0], axis=1)
    df.loc[1:,"systems_location"] = df["memilio_location"].iloc[1:].apply(lambda x: get_systems_location(df_locations, x))
    
    mean_visitors_by_station = []
    for station in range(1, 17):
        upstream_locations = upstream_locations_dict[str(station)]
        df_subset = df.loc[df.systems_location.isin(upstream_locations), :]
        visitor_list = df_subset.apply(lambda row: daily_agents_one_loc(row), axis=1).to_list()
        num_visitors = []
        for day in range(0, 14):
            visitors = [item[day] for item in visitor_list]
            visitors = [x for xs in visitors for x in xs]
            visitors = list(set(visitors))
            num_visitors.append(len(visitors))
        mean_visitors_by_station.append(statistics.mean(num_visitors))
    
    results.append(mean_visitors_by_station)
    print(sim)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [10]:
df_results = pd.DataFrame(results)
print(df_results.mean(axis = 0).round())

0      92.0
1      49.0
2     157.0
3     304.0
4     264.0
5     184.0
6     674.0
7     689.0
8      97.0
9     140.0
10    191.0
11    745.0
12     78.0
13    774.0
14    391.0
15    871.0
dtype: float64
